#  Simulation Coursework Part 1

In this problem we will suppose that S satifies the standard Geometric Brownian Motion and consider a down-and-out barrier option. We will then implement the pathwise sensitivity method to estimate **Delta** and **Vega** for this option and achieve $O(h)$ weak convergence.


Consider the following stochastic differential equation:
 $$
 dS_t =rS_t dt+\sigma S_t dW_t, \quad t \in [0,T];   \space S_0=s_0.
 $$

Consider a down-and-out barrier option with the discounted payoff:
$$
V= \mathbb{E}\left[e^{-rT} (S_T-K)^+1_{\min_{t\in [0,T]}S_t>B}\right],
$$
with a barrier $0<B<K$.

In this question, we take the parameter values  $r=0.05$, $\sigma=0.5$, $T=1$, $s_0=100$, $K=110$ and $B=90$.



Firstly, we import all modules that we need and seed the random number generator.

In [143]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [145]:
np.random.seed(0)

In [147]:
# Problem parameters
r = 0.05
sigma = 0.5
S0 = 100
K=110
B=90
T=1

### Barrier Crossing Technique

The **barrier crossing technique** is a method used to correct the naive approach of checking $S_t > B$ only at discrete time steps in Monte Carlo simulations. Since simulating $S_t$ at discrete time points $t_0, t_1, \dots, t_N$ ignores potential crossings between time steps, the Brownian Bridge correction estimates the probability that the process stays above the barrier in each interval.

#### **1. Brownian Bridge Survival Probability**
For each interval $[t_i, t_{i+1}]$, given $S_{t_i}$ and $S_{t_{i+1}}$, the probability that $S_t$ **does not cross** $B$ in $[t_i, t_{i+1}]$ is:

$$
P_i = 1 - \exp\left( -\frac{2 \ln(S_{t_i} / B) \ln(S_{t_{i+1}} / B)}{\sigma^2 \Delta t} \right),
$$

where $\Delta t = t_{i+1} - t_i$.

#### **Derivation**
Define the log-price process:
$$
X_t = \ln S_t.
$$

Since $S_t$ follows **Geometric Brownian Motion (GBM)**:
$$
dS_t = r S_t dt + \sigma S_t dW_t,
$$

the transformed process $X_t = \ln S_t$ follows **arithmetic Brownian motion**:
$$
dX_t = \left( r - \frac{\sigma^2}{2} \right) dt + \sigma dW_t.
$$

The probability that $X_t$ stays above $\ln B$ in $[t_i, t_{i+1}]$ is equivalent to a **Brownian Bridge staying above the barrier**. 
Using the Reflection Principle, we get:

$$
P\left(\min_{t \in [t_i, t_{i+1}]} X_t > \ln B \mid X_{t_i}, X_{t_{i+1}} \right) = 
1 - \exp\left(-\frac{2 (X_{t_i} - \ln B)(X_{t_{i+1}} - \ln B)}{\sigma^2 \Delta t} \right).
$$

Substituting $X_{t_i} = \ln S_{t_i}$ and simplifying, we obtain the formula for $P_i$.

#### **2. Total Survival Probability**
The probability that $S_t$ **never** crosses $B$ in the entire time interval $[0, T]$ is the **product of all interval probabilities**:

$$
Q = \prod_{i=0}^{N-1} P_i.
$$

#### **3. Computing the Option Price**
Since the down-and-out option has a **discontinuous payoff**, the barrier crossing technique described in **Section 6.4 of Glasserman's textbook** replaces the discontinuous indicator function in the payoff computation with the smooth survival probability function $Q$, which is differentiable.

Instead of computing:

$$
\mathbb{E} \left[ e^{-rT} (S_T-K)^+ \cdot \mathbf{1}{\{\min_{t \in [0,T]} S_t > B\}} \right],
$$

the method evaluates:

$$
\mathbb{E} \left[ e^{-rT} (S_T-K)^+ \cdot Q) \right].
$$

where $(S_T-K)^+$ represents the option's intrinsic value.

By replacing the hard indicator function with the smooth barrier crossing probability, this approach enables differentiability and smoothness, which allows to compute Delta and Vega using pathwise sensitivity method.

The following code simulates the asset price paths using GBM formula and implements the barrier crossing technique.

In [151]:
N = 252          # Number of time steps
M = 100000       # Number of simulated paths
dt = T / N

# Simulate GBM paths
Z = np.random.normal(size=(M, N))
X = np.zeros((M, N+1))
X[:, 0] = np.log(S0)

for i in range(N):
    X[:, i+1] = X[:, i] + (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, i]

S = np.exp(X)

# Check if all Si > B for each path i
all_above_B = np.all(S > B, axis=1)

# Compute P_i for each interval
P = 1 - np.exp(-2 * np.log(S / B)[:, :-1] * np.log(S / B)[:, 1:] / (sigma**2 * dt))

# Compute total survival probability Q for each path
Q = np.zeros(M)
Q[all_above_B] = np.prod(P[all_above_B, :], axis=1)

### Implement Pathwise Sensitivit to estimate Delta and Vega

To compute Greeks, we differentiate $V$ with respect to parameters $S_0$ (Delta) and $\sigma$ (Vega). Using the pathwise derivative method:

#### **Delta ($\Delta = \partial V / \partial S_0$):**
$$
\Delta =
\mathbb{E} \left[
e^{-rT} \left(
\frac{\partial (S_T - K)^+}{\partial S_0} Q
+ (S_T - K)^+ \frac{\partial Q}{\partial S_0}
\right) \right].
$$

- **Term 1:**
$$
\frac{\partial (S_T - K)^+}{\partial S_0} =
\mathbf{1}{\{S_T > K\}} \frac{\partial S_T}{\partial S_0}.
$$

- **Term 2:**
$$
\frac{\partial Q}{\partial S_0} =
Q \sum_{i=0}^{N-1} \frac{\partial \ln P_i}{\partial S_0}.
$$

#### **Vega ($V = \partial V / \partial \sigma$):**
$$
V =
\mathbb{E} \left[
e^{-rT} \left(
\frac{\partial (S_T - K)^+}{\partial \sigma} Q
+ (S_T - K)^+ \frac{\partial Q}{\partial \sigma}
\right) \right].
$$

- **Term 1:**
$$
\frac{\partial (S_T - K)^+}{\partial \sigma} =
\mathbf{1}{\{S_T > K\}} \frac{\partial S_T}{\partial \sigma}.
$$

- **Term 2:**
$$
\frac{\partial Q}{\partial \sigma} =
Q \sum_{i=0}^{N-1} \frac{\partial \ln P_i}{\partial \sigma}.
$$

---

#### **Derivative of $S_T$:**
For **GBM**, the terminal price is:
$$
S_T = S_0 \exp\left( \left( r - \frac{\sigma^2}{2} \right) T + \sigma W_T \right).
$$
Taking derivatives:
$$
\frac{\partial S_T}{\partial S_0} = \frac{S_T}{S_0},
\quad
\frac{\partial S_T}{\partial \sigma} = S_T \left(-\sigma T + W_T \right).
$$

#### **Derivative of $\ln P_i$:**
Given:
$$
P_i = 1 - \exp\left( -\frac{2 \ln(S_i / B) \ln(S_{i+1} / B)}{\sigma^2 \Delta t} \right),
$$
we define:
$$
\ln P_i = \ln \left( 1 - \exp\left( -\frac{2 A_i}{\sigma^2 \Delta t} \right) \right),
$$
where:
$$
A_i = \ln(S_i / B) \ln(S_{i+1} / B).
$$

- **For $\frac{\partial \ln P_i}{\partial S_0}$:**
$$
\frac{\partial \ln P_i}{\partial S_0} =
\frac{\exp\left( -\frac{2 A_i}{\sigma^2 \Delta t} \right)}{P_i}
\cdot \frac{2 ( \ln(S_i / B) + \ln(S_{i+1} / B) )}{\sigma^2 \Delta t S_0}.
$$

- **For $\frac{\partial \ln P_i}{\partial \sigma}$:**
$$
\frac{\partial \ln P_i}{\partial \sigma} =
\frac{\exp\left( -\frac{2 A_i}{\sigma^2 \Delta t} \right)}{P_i}
\cdot \frac{4 A_i}{\sigma^3 \Delta t}.
$$


In [155]:
# Calculate S_T and payoff
S_T = S[:, -1]
payoff = np.exp(-r * T) * np.maximum(S_T - K, 0) * Q

# Compute derivatives
dS_TdS0 = S_T / S0
W_T = (np.log(S_T / S0) - (r - 0.5 * sigma**2) * T) / sigma
dS_Tdsigma = S_T * (-sigma * T + W_T)

# Compute derivatives of Q
sum_dPdS0 = np.zeros(M)
sum_dPdsigma = np.zeros(M)

# Compute terms for survived paths
lnS_survived = np.log(S / B)[all_above_B, :]
exponent = -2 * lnS_survived[:, :-1] * lnS_survived[:, 1:] / (sigma**2 * dt)
Ai = lnS_survived[:, :-1] * lnS_survived[:, 1:]
P_survived = 1 - np.exp(exponent)

# Avoid division by zero by adding a small epsilon
epsilon = 1e-12
P_survived += epsilon

# Compute dP_i/dS0 / P_i
dPdS0_over_P = (np.exp(exponent) * (2 / (sigma**2 * dt*S0)) * (lnS_survived[:, :-1] + lnS_survived[:, 1:])) / P_survived
sum_dPdS0[all_above_B] = np.sum(dPdS0_over_P, axis=1)

# Compute dP_i/dsigma / P_i
dPdsigma_over_P = -4 * Ai * np.exp(exponent) / (sigma**3 * dt * P_survived)
sum_dPdsigma[all_above_B] = np.sum(dPdsigma_over_P, axis=1)

# Compute Delta and Vega terms
mask_payoff = (S_T > K) & (all_above_B)

Delta_term1 = np.exp(-r * T) * dS_TdS0 * mask_payoff * Q
Delta_term2 = np.exp(-r * T) * np.maximum(S_T - K, 0) * Q * sum_dPdS0
Delta = np.mean(Delta_term1 + Delta_term2)

Vega_term1 = np.exp(-r * T) * dS_Tdsigma * mask_payoff * Q
Vega_term2 = np.exp(-r * T) * np.maximum(S_T - K, 0) * Q * sum_dPdsigma
Vega = np.mean(Vega_term1 + Vega_term2)

# Output results
print(f"Estimated Delta: {Delta}")
print(f"Estimated Vega: {Vega}")

Estimated Delta: 0.849402678640391
Estimated Vega: 17.25977094640213


The barrier crossing technique achieves **$\mathcal{O}(h)$ weak convergence** by accurately estimating the continuous barrier crossing probability between discrete time steps. This is superior to naive discrete monitoring, which has an $\mathcal{O}(h)$ error.